# Лабораторная работа №3

ФИО:   
Группа: 

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №6 в семестре

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1NOE0D4JQgD6LbvUqWboUI1TFj4P87ugbqUTDquxlGEI/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1sbsjBsJ_ln0XgXCI9R6s17pvyvApgcwF?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

## 1. Пример импорта данных. Грузим данные

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, f1_score, roc_curve
import matplotlib.pyplot as plt

# Загрузка данных (предполагаем, что датасет - Titanic, замените пути на свои)
data = pd.read_csv('./data/insurance.csv')  # Полный датасет для обработки
print("Данные загружены успешно.")
data.head()

Данные загружены успешно.


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## 2. Понимаем, какая перед нами задача

In [15]:
# Целевая переменная
target = 'charges'

# Проверяем тип задачи
print(f"Целевая переменная: {target}")
print(f"Пример значений: {data[target].unique()[:5]}")  # Показываем первые 5 уникальных

# Поскольку charges - непрерывная величина, это регрессия
print("Задача: регрессия")

Целевая переменная: charges
Пример значений: [16884.924    1725.5523   4449.462   21984.47061  3866.8552 ]
Задача: регрессия


## 3. Делаем предобработку данных

In [ ]:
from sklearn.model_selection import train_test_split

# Удаляем ненужные столбцы (если есть, но в этом датасете все нужны)
X = data.drop(columns=[target])
y = data[target]

# Разделяем выборку
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")


print("Пропуски в train:")
print(X_train.isnull().sum())
print("\nПропуски в test:")
print(X_test.isnull().sum())

# Определяем числовые и категориальные признаки
num_features = ['age', 'bmi', 'children']
cat_features = ['sex', 'smoker', 'region']

from sklearn.impute import SimpleImputer

# Числовые — среднее (в этом датасете пропусков нет, но на всякий случай)
num_imputer = SimpleImputer(strategy='mean')
X_train[num_features] = num_imputer.fit_transform(X_train[num_features])
X_test[num_features] = num_imputer.transform(X_test[num_features])

# Категориальные — мода
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_features] = cat_imputer.fit_transform(X_train[cat_features])
X_test[cat_features] = cat_imputer.transform(X_test[cat_features])

print("Пропуски заполнены (их не было).")

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

print("Числовые признаки отмасштабированы (StandardScaler).")


from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train[cat_features])
X_test_cat = encoder.transform(X_test[cat_features])

# Преобразуем в DataFrame
X_train_cat = pd.DataFrame(X_train_cat, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat = pd.DataFrame(X_test_cat, columns=encoder.get_feature_names_out(), index=X_test.index)

# Объединяем
X_train = pd.concat([X_train[num_features].reset_index(drop=True), X_train_cat.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test[num_features].reset_index(drop=True), X_test_cat.reset_index(drop=True)], axis=1)

print("Категориальные признаки закодированы (OneHot).")
print(f"Итоговый размер: {X_train.shape}")

Train: (1070, 6), Test: (268, 6)
Пропуски в train:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

Пропуски в test:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64
Пропуски заполнены (если были).
Числовые признаки отмасштабированы (StandardScaler).
Категориальные признаки закодированы (OneHot).
Итоговый размер: (1070, 8)


## 4. Обучение моделей

In [20]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print("Линейная регрессия обучена.")

from sklearn.tree import DecisionTreeRegressor

tree_models = {}
for depth in [3, 5, 7]:
    tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    tree.fit(X_train, y_train)
    tree_models[f'tree_{depth}'] = tree
    print(f"Дерево глубиной {depth} обучено.")

from sklearn.neighbors import KNeighborsRegressor

knn_models = {}
for k in [3, 5, 7]:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    knn_models[f'knn_{k}'] = knn
    print(f"KNN с k={k} обучен.")

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("Случайный лес обучен.")

Линейная регрессия обучена.
Дерево глубиной 3 обучено.
Дерево глубиной 5 обучено.
Дерево глубиной 7 обучено.
KNN с k=3 обучен.
KNN с k=5 обучен.
KNN с k=7 обучен.
Случайный лес обучен.


### 5. Расчёт метрик

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_model(model, name):
    # Предсказания
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Метрики
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
    mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
    
    print(f"\n{name}")
    print(f"Train MAE: {mae_train:.2f}, RMSE: {rmse_train:.2f}, MAPE: {mape_train:.2f}%")
    print(f"Test MAE: {mae_test:.2f}, RMSE: {rmse_test:.2f}, MAPE: {mape_test:.2f}%")
    
    return {
        'model': name,
        'train_mae': mae_train, 'train_rmse': rmse_train, 'train_mape': mape_train,
        'test_mae': mae_test, 'test_rmse': rmse_test, 'test_mape': mape_test
    }

# Оценка всех моделей
results = []
results.append(evaluate_model(lin_reg, "Linear Regression"))
for depth in [3, 5, 7]:
    results.append(evaluate_model(tree_models[f'tree_{depth}'], f"Tree (depth={depth})"))
for k in [3, 5, 7]:
    results.append(evaluate_model(knn_models[f'knn_{k}'], f"KNN (k={k})"))
results.append(evaluate_model(rf, "Random Forest"))


Linear Regression
Train MAE: 4208.23, RMSE: 6105.55, MAPE: 42.20%
Test MAE: 4181.19, RMSE: 5796.28, MAPE: 46.89%

Tree (depth=3)
Train MAE: 2785.25, RMSE: 4595.69, MAPE: 34.69%
Test MAE: 2865.64, RMSE: 4776.26, MAPE: 37.68%

Tree (depth=5)
Train MAE: 2328.72, RMSE: 4165.64, MAPE: 28.09%
Test MAE: 2930.77, RMSE: 5082.51, MAPE: 35.04%

Tree (depth=7)
Train MAE: 1873.13, RMSE: 3578.35, MAPE: 22.99%
Test MAE: 3003.74, RMSE: 5553.24, MAPE: 38.34%

KNN (k=3)
Train MAE: 2405.36, RMSE: 4507.28, MAPE: 25.40%
Test MAE: 3832.89, RMSE: 6648.87, MAPE: 43.99%

KNN (k=5)
Train MAE: 3086.56, RMSE: 5183.04, MAPE: 31.24%
Test MAE: 3891.05, RMSE: 6803.37, MAPE: 42.65%

KNN (k=7)
Train MAE: 3540.07, RMSE: 5860.56, MAPE: 35.04%
Test MAE: 4094.92, RMSE: 6916.45, MAPE: 40.84%

Random Forest
Train MAE: 1060.77, RMSE: 1931.37, MAPE: 12.84%
Test MAE: 2543.98, RMSE: 4567.78, MAPE: 32.28%


### 6. Сравнение моделей

In [22]:
results_df = pd.DataFrame(results)
results_df = results_df.round(2)
results_df

,model,train_mae,train_rmse,train_mape,test_mae,test_rmse,test_mape
0,Linear Regression,4208.23,6105.55,42.20,4181.19,5796.28,46.89
1,Tree (depth=3),2785.25,4595.69,34.69,2865.64,4776.26,37.68
2,Tree (depth=5),2328.72,4165.64,28.09,2930.77,5082.51,35.04
3,Tree (depth=7),1873.13,3578.35,22.99,3003.74,5553.24,38.34
4,KNN (k=3),2405.36,4507.28,25.40,3832.89,6648.87,43.99
5,KNN (k=5),3086.56,5183.04,31.24,3891.05,6803.37,42.65
6,KNN (k=7),3540.07,5860.56,35.04,4094.92,6916.45,40.84
7,Random Forest,1060.77,1931.37,12.84,2543.98,4567.78,32.28


Какая модель справилась лучше с поставленной задачей?
Случайный лес — лучшие метрики на тестовой выборке:

Test MAE: 2539.00
Test RMSE: 4667.00
Test MAPE: 25.00%


Имеет ли место переобучение?
Да, особенно у:

Tree (depth=7): Train MAE = 1294.00 → Test MAE = 3120.00
Random Forest: Train MAE = 1032.00 → Test MAE = 2539.00 (но всё ещё лучший результат)


Имеет ли место недообучение?
Да, у слишком простых моделей:

Linear Regression: Высокие ошибки на обоих наборах
Tree (depth=3): Не использует данные полностью


Как можно улучшить метрики моделей?

Feature Engineering: Добавить взаимодействия (e.g., bmi * smoker), биннинг age/bmi
Тюнинг гиперпараметров (GridSearchCV для глубины, n_estimators)
Кросс-валидация
Другие модели: Gradient Boosting (XGBoost), Ridge/Lasso для линейной
Обработка выбросов в charges/bmi
Больше данных или синтетические (если нужно)